In [24]:
import pandas as pd
import fasttext
import re
from datetime import timedelta

## Data Preprocessing

1. Identification and merging of relevant messages

In [25]:
# Read contents files
df_2023_contents = pd.read_csv("~/Desktop/Tickets/2023/contents.csv")
df_2023_contents = pd.read_csv("~/Desktop/Tickets/2023/contents.csv")
df_2021_contents = pd.read_csv("~/Desktop/Tickets/2021/contents.csv")
df_2022_contents = pd.read_csv("~/Desktop/Tickets/2022/contents.csv", on_bad_lines='skip')
df_2020_contents = pd.read_csv("~/Desktop/Tickets/2020/contents.csv")
df_2019_contents = pd.read_csv("~/Desktop/Tickets/2019/contents.csv")
df_2018_contents = pd.read_csv("~/Desktop/Tickets/2018/contents.csv")
df_2017_contents = pd.read_csv("~/Desktop/Tickets/2017/contents.csv")
df_2016_contents = pd.read_csv("~/Desktop/Tickets/2016/contents.csv")
df_2015_contents = pd.read_csv("~/Desktop/Tickets/2015/contents.csv")

# Merge content information
df_result = pd.concat([df_2015_contents,df_2016_contents,df_2017_contents,df_2018_contents,df_2019_contents,df_2020_contents,df_2021_contents,df_2022_contents,df_2023_contents])
df_result = df_result.map(lambda x: x.strip() if isinstance(x, str) else x)

In [26]:
# Create timestamp column
df_result['timestamp'] = df_result['Datum'] + ' ' + df_result['Uhrzeit']
df_result['timestamp'] = pd.to_datetime(df_result['timestamp'])
df_result = df_result.drop(columns=['Datum', 'Uhrzeit'])

In [27]:
# Only keep the first message of a ticket and all messages sent within the first 3 minutes.
def filter_group(group):
    min_time = group['timestamp'].min()
    threshold_time = min_time + timedelta(minutes=3)
    return group[group['timestamp'] <= threshold_time]

df_result = df_result.groupby('ID', group_keys=False).apply(filter_group)

/tmp/ipykernel_7579/521410608.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_result = df_result.groupby('ID', group_keys=False).apply(filter_group)


In [28]:
# Exclude irrelevant messages
df_result = df_result[~((df_result['Nachrichtentyp'] == 'Lösung') | (df_result['Nachrichtentyp'] == 'Interne Notiz') | (df_result['Nachrichtentyp'] == 'Systemdaten'))]

# Only keep U-Users and H-Users 
df_result = df_result[df_result['Absender'].str.startswith(('H', 'U') , na=False)]

In [29]:
# Merge ticket texts
def merge_rows(group):
    merged_text = ' '.join(group['Text'])
    row_with_smallest_timestamp = group.loc[group['timestamp'].idxmin()].copy()
    row_with_smallest_timestamp['Text'] = merged_text
    return row_with_smallest_timestamp

df_result = df_result.groupby('ID').apply(merge_rows).reset_index(drop=True)

/tmp/ipykernel_7579/771059327.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_result = df_result.groupby('ID').apply(merge_rows).reset_index(drop=True)


2. Adding ticket information

In [30]:
# Read tickets files
df_2023_tickets = pd.read_csv("~/Desktop/Tickets/2023/tickets.csv")
df_2021_tickets = pd.read_csv("~/Desktop/Tickets/2021/tickets.csv")
df_2022_tickets = pd.read_csv("~/Desktop/Tickets/2022/tickets.csv")
df_2020_tickets = pd.read_csv("~/Desktop/Tickets/2020/tickets.csv")
df_2019_tickets = pd.read_csv("~/Desktop/Tickets/2019/tickets.csv")
df_2018_tickets = pd.read_csv("~/Desktop/Tickets/2018/tickets.csv")
df_2017_tickets = pd.read_csv("~/Desktop/Tickets/2017/tickets.csv")
df_2016_tickets = pd.read_csv("~/Desktop/Tickets/2016/tickets.csv")
df_2015_tickets = pd.read_csv("~/Desktop/Tickets/2015/tickets.csv")

# Merge ticket information
df_tickets = pd.concat([df_2015_tickets,df_2016_tickets,df_2017_tickets,df_2018_tickets,df_2019_tickets,df_2020_tickets,df_2021_tickets,df_2022_tickets,df_2023_tickets])
df_tickets = df_tickets.map(lambda x: x.strip() if isinstance(x, str) else x)

In [31]:
# Inner join
df_result = pd.merge(df_result, df_tickets, on='ID', how='inner')

In [32]:
# Read file from Janik to add verified ticket labels
df_labels = pd.read_csv("~/Desktop/Tickets/ticket_labels_JSc.csv")
df_labels = df_labels.map(lambda x: x.strip() if isinstance(x, str) else x)
df_labels.drop(index=[7531,7532,7533,7534], inplace=True)
df_labels['ID'] = df_labels['ID'].astype('int')

In [33]:
# Left outer join
df_result = pd.merge(df_result, df_labels, on='ID', how='left')

In [34]:
df_result = df_result.drop(df_result[df_result['support_level'] == 'Not relevant'].index)

In [35]:
# Ensure consistency of label values

# support_level
mapping = {'1st Level': 1, '2nd Level': 2}
df_result['support_level'] = df_result['support_level'].replace(mapping)

# Ticket Label
mapping = {'': None, '1. Level': 1, '2. Level': 2}
df_result['Ticket Label'] = df_result['Ticket Label'].replace(mapping)

# Abteilung Label
mapping = {'Vertrag': 'Contract', 'Applikation': 'Application', '': None}
df_result['Abteilung Label'] = df_result['Abteilung Label'].replace(mapping)

# Product
mapping = {'': None, 'Business By Design': 'Business by Design', 'Entwicklungssystem bzw. Mandant': 'Entwicklungssystem/Mandant'}
df_result['Produkt Label'] = df_result['Produkt Label'].replace(mapping)

/tmp/ipykernel_7579/3762762705.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_result['support_level'] = df_result['support_level'].replace(mapping)
/tmp/ipykernel_7579/3762762705.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_result['Ticket Label'] = df_result['Ticket Label'].replace(mapping)


In [36]:
# Create a mask where all specified columns are NaN
mask = df_result['support_level'].isna() & df_result['department'].isna() & df_result['product'].isna()

# Use loc to target the rows and columns where the mask is True and replace them
df_result.loc[mask, 'support_level'] = df_result.loc[mask, 'Ticket Label']
df_result.loc[mask, 'department'] = df_result.loc[mask, 'Abteilung Label']
df_result.loc[mask, 'product'] = df_result.loc[mask, 'Produkt Label']

# Drop irrelevant columns
df_result = df_result.drop(columns=['Ticket Label', 'Abteilung Label', 'Produkt Label'])

3. Merging the ticket text with the description

In [37]:
# Merge text and description into text
df_result['Text'] = df_result['Text'].str.cat(df_result['Beschreibung'], sep=' ')
df_result = df_result.drop(columns=['Beschreibung'])

4. Data cleaning

In [38]:
# Drop rows
df_result = df_result[~((df_result['support_level'].isna()) & df_result['department'].isna() & df_result['product'].isna())]
df_result = df_result[~((df_result['support_level'] == 2) & df_result['department'].isna() & df_result['product'].isna())]
df_result = df_result[~((df_result['support_level'] == 2) & (df_result['department'] == 'Application') & df_result['product'].isna())] # new
df_result = df_result[~((df_result['support_level'] == 2) & (df_result['department'] == 'Basis') & df_result['product'].isna())] # new

In [39]:
# Replace with none
mask = (df_result['support_level'] == 1) & ((df_result['department'].notna()) | (df_result['product'].notna()))
df_result.loc[mask, ['department', 'product']] = None

mask = (df_result['support_level'] == 2) & (df_result['department'] == 'Contract') & (df_result['product'].notna()) # new
df_result.loc[mask, ['department', 'product']] = None # new

In [40]:
# Remove '\n' symbols from text
df_result['Text'] = df_result['Text'].str.replace('\n', ' ')

# Convert the text to lowercase
df_result['Text'] = df_result['Text'].str.lower()

# Remove URLs
df_result['Text'] = df_result['Text'].str.replace(r'(http[s]?://|www\.)(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '<url>', regex=True)

# Remove email addresses
df_result['Text'] = df_result['Text'].str.replace(r'\S+@\S+', '<email>', regex=True)

# Strip leading and trailing white spaces from the text
df_result['Text'] = df_result['Text'].str.strip()

# Remove extra spaces from the text
df_result['Text'] = df_result['Text'].str.replace(r'\s+', ' ', regex=True)

In [41]:
# Drop irrelevant columns and rename columns
df_result = df_result[['ID','Absender','Text','Kategorie ID','Unterkategorie ID','timestamp','support_level','department','product']]
df_result = df_result.rename(columns={'ID': 'id', 'Absender':'sender', 'Text': 'text','Beschreibung': 'description','Kategorie ID':'category','Unterkategorie ID':'subcategory','support_level':'level'})

In [42]:
df_result.to_csv('~/Desktop/Tickets/data_processed.csv', index=False)

Validations

In [43]:
df_result['level'].value_counts(dropna=True)

level
2.0    5387
1.0    2513
Name: count, dtype: int64

In [44]:
# Create a mask where 'support_level' equals 2, 'department' equals 'Contract', and 'product' is NaN
mask = (df_result['level'] == 2) & (df_result['department'] == 'Application') & (df_result['product'].isna())

# Count the number of rows where the mask is True
count_rows = mask.sum()

print(count_rows)

0


## Language Detection (FastText)

In [45]:
model = fasttext.load_model('lid.176.ftz')

def detect_language_with_probability(text, threshold=0.7):
    predictions = model.predict(text, k=1)
    language, probability = predictions[0][0], predictions[1][0]
    
    if probability >= threshold:
        return (language.replace('__label__', ''), probability)  # Extract language code and probability
    else:
        return ('unknown', probability)

df_result['language_probability'] = df_result['text'].apply(detect_language_with_probability)
df_result[['language', 'probability']] = pd.DataFrame(df_result['language_probability'].tolist(), index=df_result.index)

df_result.drop(columns=['language_probability'], inplace=True)
df_result['language'].value_counts()

ValueError: lid.176.ftz cannot be opened for loading!

## Strategies ##

In [ ]:
# Remove names
def remove_names(row):
    # Split the 'sender' to extract names
    parts = row['sender'].split()
    surname, lastname = parts[1], parts[2]
    
    surname_pattern = re.compile(re.escape(surname), re.IGNORECASE)
    lastname_pattern = re.compile(re.escape(lastname), re.IGNORECASE)
    
    cleaned_text = surname_pattern.sub('<surname>', row['text'])
    cleaned_text = lastname_pattern.sub('<lastname>', cleaned_text)
    
    return cleaned_text

df_result['text'] = df_result.apply(remove_names, axis=1)

In [ ]:
# Merge GBI and GBS
merged_category = 'Global Bike'
categories_to_merge = ['GBI', 'GBS/Digital Transformation Curriculum']
df_result['product'] = df_result['product'].replace(categories_to_merge, merged_category)
df_result['product'].value_counts()

product
Global Bike                                  1712
ERPsim                                        639
IDES                                          453
HANA                                          414
Sonstiges                                     392
SAP Business Warehouse & Business Objects     311
Entwicklungssystem/Mandant                    196
TS410                                         155
SAP4School                                    112
Business by Design                             37
UCC Portal                                     30
Lumira                                         23
Celonis                                        10
UCC Hardware                                    1
Name: count, dtype: int64

In [ ]:
# Remove legacy products
values_to_drop = ['Lumira', 'Celonis', 'IDES'] 
df_result = df_result[~df_result['product'].isin(values_to_drop)]
df_result['product'].value_counts()

product
Global Bike                                  1712
ERPsim                                        639
HANA                                          414
Sonstiges                                     392
SAP Business Warehouse & Business Objects     311
Entwicklungssystem/Mandant                    196
TS410                                         155
SAP4School                                    112
Business by Design                             37
UCC Portal                                     30
Name: count, dtype: int64

In [ ]:
# Remove Sonstiges class
values_to_drop = ['Sonstiges'] 
df_result = df_result[~df_result['product'].isin(values_to_drop)]
df_result['product'].value_counts()

product
Global Bike                                  1712
ERPsim                                        639
HANA                                          414
SAP Business Warehouse & Business Objects     311
Entwicklungssystem/Mandant                    196
TS410                                         155
SAP4School                                    112
Business by Design                             37
UCC Portal                                     30
Name: count, dtype: int64